# OpenAI Introduction

Use this for [reference](https://platform.openai.com/docs/api-reference/authentication)

In [1]:
%pipenv install openai python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


> Create an OpenAI account and get an API_KEY

### OpenAI:

1. Chat completion API ✅
2. Implement structured output ✅
3. Implement tool calling ✅
4. Implement streaming ✅
5. Implement assistant
6. Response API
7. Chaining of response

In [60]:
from openai import OpenAI
from os import getenv
from dotenv import load_dotenv

load_dotenv()

True

In [61]:
client = OpenAI()

### List Models

In [14]:
models = client.models.list()
model_names = [model.id for model in models if "gpt" in model.id]

In [15]:
model_names

['gpt-4o-audio-preview-2024-10-01',
 'gpt-4o-mini-audio-preview',
 'gpt-4o-audio-preview',
 'gpt-4.1-nano',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-4o-mini-search-preview',
 'gpt-4.1-nano-2025-04-14',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-1106',
 'gpt-4o-search-preview',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo',
 'gpt-4o-mini-search-preview-2025-03-11',
 'gpt-4o-2024-11-20',
 'gpt-4o-2024-05-13',
 'gpt-4o-mini-tts',
 'gpt-4o-transcribe',
 'gpt-4.5-preview',
 'gpt-4.5-preview-2025-02-27',
 'gpt-4o-search-preview-2025-03-11',
 'gpt-image-1',
 'gpt-4o',
 'gpt-4o-2024-08-06',
 'gpt-4o-mini-2024-07-18',
 'gpt-4.1-mini',
 'gpt-4o-mini',
 'gpt-4o-mini-audio-preview-2024-12-17',
 'gpt-3.5-turbo-0125',
 'gpt-4o-mini-transcribe',
 'gpt-4.1-mini-2025-04-14',
 'gpt-4.1',
 'gpt-4.1-2025-04-14']

| 1. Chat Completion |
|--|


The Chat Completions API endpoint will generate a model response from a list of messages comprising a conversation.

In [62]:
MODEL_NAME = "gpt-4o-mini"

In [63]:
completion = client.chat.completions.create(
  model=MODEL_NAME,
  store=False,
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

In [64]:
print(completion.choices[0].message.content)

Lines of code converge,  
Whispers of thought and machine—  
A mind yet to learn.


| 2. Structured Output |
|--|

Structured Outputs is a feature that ensures the model will always generate responses that adhere to your supplied JSON Schema,

In [ ]:
!pipenv install pydantic

In [65]:
from pydantic import BaseModel

In [66]:
class UserRegistration(BaseModel):
    name: str
    age: int
    tier: int

In [67]:
completion = client.beta.chat.completions.parse(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Extract the user information."},
        {"role": "user", "content": "We have a new user named Tim aged 25 with a tier 3 account."},
    ],
    response_format=UserRegistration,
)

In [68]:
event = completion.choices[0].message.parsed
event

UserRegistration(name='Tim', age=25, tier=3)

| 3. Tool/Function Calling |
|--|

Function calling provides a powerful and flexible way for OpenAI models to interface with your code or external services. 



In [69]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

| Function -> Tool JSON Convertor |
|--|

Automate tool JSON creation

In [70]:
import inspect
from typing import Callable, get_type_hints

In [71]:
def generate_json_schema(func: Callable) -> str:
    """
    Generate a JSON schema for a given function based on its docstring and type hints.

    Args:
        func (Callable): The Python function to generate a schema for.

    Returns:
        str: A JSON string representing the function's schema.

    Raises:
        ValueError: If the function lacks a docstring.
    """
    # Get function metadata
    if not func.__doc__:
        raise ValueError("Function must have a docstring")

    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    # Parse docstring
    docstring = inspect.getdoc(func) or ""
    doc_lines = docstring.split('\n')
    description = doc_lines[0].strip()  # First line is the main description
    param_descriptions = {}
    in_args_section = False

    # Extract parameter descriptions
    for line in doc_lines:
        line = line.strip()
        if line.lower().startswith('args:'):
            in_args_section = True
            continue
        if in_args_section and line and ':' in line:
            # Handle lines like "param_name (type): description"
            parts = line.split(':', 1)
            param_part = parts[0].strip()
            desc = parts[1].strip() if len(parts) > 1 else ""
            # Extract param_name from "param_name (type)"
            if '(' in param_part and ')' in param_part:
                param_name = param_part[:param_part.index('(')].strip()
            else:
                param_name = param_part
            if param_name:
                param_descriptions[param_name] = desc

    # Map Python types to JSON schema types
    type_mapping = {
        'str': 'string',
        'int': 'integer',
        'float': 'number',
        'bool': 'boolean',
        'dict': 'object',
        'list': 'array'
    }

    # Build schema
    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": description,
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
                "additionalProperties": False
            },
            "strict": True
        }
    }

    # Add parameters to schema
    for param_name, param in signature.parameters.items():
        # Get type from type hints, default to str if not specified
        param_type = type_hints.get(param_name, str).__name__
        schema_type = type_mapping.get(param_type, 'string')  # Default to string
        
        schema["function"]["parameters"]["properties"][param_name] = {
            "type": schema_type,
            "description": param_descriptions.get(param_name, f"{param_name} parameter")
        }
        # Mark as required if no default value
        if param.default == inspect.Parameter.empty and param.kind != inspect.Parameter.VAR_KEYWORD:
            schema["function"]["parameters"]["required"].append(param_name)

    return schema

#### Weather function

In [72]:
def get_weather(location: str) -> dict:
    """
    Get current temperature for a given location.

    Args:
        location (str): City and country e.g. Bogotá, Colombia

    Returns:
        dict: A dictionary containing the current temperature and other weather details.
    """
    return {"location": location, "temperature": 25, "unit": "Celsius"}

In [73]:
generate_json_schema(func=get_weather)

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get current temperature for a given location.',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'}},
   'required': ['location'],
   'additionalProperties': False},
  'strict': True}}

In [74]:
tools[0]

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get current temperature for a given location.',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'City and country e.g. Bogotá, Colombia'}},
   'required': ['location'],
   'additionalProperties': False},
  'strict': True}}

#### Tool Calling

In [75]:
completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

In [76]:
print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_I2B8aQ6ASMD15foX78hyFOvw', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]


| 4. Streaming |
|--|


Streaming responses lets you start printing or processing the beginning of the model's output while it continues generating the full response.

In [77]:
stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {
            "role": "user",
            "content": "Say 'double bubble bath' ten times fast.",
        },
    ],
    stream=True,
)

In [78]:
for chunk in stream:
    print(chunk)
    print(chunk.choices[0].delta)
    print("****************")

ChatCompletionChunk(id='chatcmpl-BbiPpqPC2l2GD2pCMomeKBZR2ByJ9', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1748328569, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_54eb4bd693', usage=None)
ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None)
****************
ChatCompletionChunk(id='chatcmpl-BbiPpqPC2l2GD2pCMomeKBZR2ByJ9', choices=[Choice(delta=ChoiceDelta(content='Sure', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1748328569, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_54eb4bd693', usage=None)
ChoiceDelta(content='Sure', function_call=None, refusal=None, role=None, tool_calls=None)
****************
ChatCompletionChunk(

| Assistants |
|--|

Build assistants that can call models and use tools to perform tasks.

In [80]:
my_assistant = client.beta.assistants.create(
    instructions="You are a personal math tutor. When asked a question, write and run Python code to answer the question.",
    name="Math Tutor",
    tools=[{"type": "code_interpreter"}],
    model=MODEL_NAME
)

In [85]:
print(my_assistant.id)

asst_6gzaY6WEnSo3sEaWiHDfkv49


In [82]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_6gzaY6WEnSo3sEaWiHDfkv49', created_at=1748328593, description=None, instructions='You are a personal math tutor. When asked a question, write and run Python code to answer the question.', metadata={}, model='gpt-4o-mini', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)]


In [86]:
my_assistant = client.beta.assistants.retrieve(assistant_id=my_assistant.id)

In [87]:
my_assistant.id

'asst_6gzaY6WEnSo3sEaWiHDfkv49'

Create a Thread

In [88]:
thread = client.beta.threads.create()
thread_id = thread.id
print(f"Thread ID: {thread_id}")

Thread ID: thread_NuxfP59uuYhnz5uZCmukGmgV


In [89]:
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content="Explain factorial?"
)

In [90]:
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=my_assistant.id
)

# Wait for the run to complete
import time
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
    if run_status.status == "completed":
        break
    time.sleep(1)

In [91]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
for message in messages.data:
    if message.role == "assistant":
        print(f"Assistant: {message.content[0].text.value}")

Assistant: A factorial, denoted by an exclamation mark (e.g., \( n! \)), is a mathematical operation that multiplies a positive integer \( n \) by all of the positive integers less than it down to 1. 

The factorial of a non-negative integer \( n \) can be defined as follows:

- If \( n = 0 \), then \( 0! = 1 \) (by definition).
- If \( n > 0 \), then \( n! = n \times (n-1) \times (n-2) \times \ldots \times 2 \times 1 \).

For example:
- \( 5! = 5 \times 4 \times 3 \times 2 \times 1 = 120 \)
- \( 3! = 3 \times 2 \times 1 = 6 \)

Factorials are commonly used in permutations, combinations, and other areas of mathematics, particularly in statistics and probability.

Would you like to see how to calculate factorials using Python?


| 6. Response API |
|--|

Allow the model access to external systems and data using function calling.

In [92]:
response = client.responses.create(
    model=MODEL_NAME,
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

As the moon lit up the enchanted forest, Luna the unicorn spread her shimmering wings and soared through the starlit sky, painting dreams for all the sleeping children below.


Response API with Tools

In [93]:
response = client.responses.create(
    model=MODEL_NAME,
    tools=[{"type": "web_search_preview"}],
    input="What was a positive news story from today?"
)

print(response.output_text)

As of May 27, 2025, several positive environmental developments have been reported:

1. **U.S. Introduces Limits on PFAS in Drinking Water**: The Environmental Protection Agency (EPA) has set its first-ever national legally enforceable limits on per- and polyfluoroalkyl substances (PFAS) in drinking water. This regulation aims to reduce PFAS exposure for approximately 100 million people by requiring public water systems to decrease contamination by 2029. ([homeplanet.grove.co](https://homeplanet.grove.co/blog-posts/positive-environmental-news-stories-that-give-us-hope-in-2025?utm_source=openai))

2. **EU's Nature Restoration Law**: The European Union has enacted the Nature Restoration Law, a comprehensive initiative to restore damaged ecosystems and boost biodiversity. The law targets restoring 20% of the EU's land and sea areas by 2030 and all degraded ecosystems by 2050. ([homeplanet.grove.co](https://homeplanet.grove.co/blog-posts/positive-environmental-news-stories-that-give-us-hop